**Plot histograms in a loop, making a plot per run**  
This allows to relatively quickly scan all runs or a collection of runs for either general characteristics of the histograms under investigation or spot anomalies by eye

In [ ]:
### imports

# external modules
import sys
import numpy as np
import matplotlib.pyplot as plt
import importlib

# internal modules
sys.path.append('utils')
import csv_utils as csvu
import hist_utils as hu
import dataframe_utils as dfu
import plot_utils as pu
import clustering_utils as cu
importlib.reload(csvu)
importlib.reload(hu)
importlib.reload(dfu)
importlib.reload(pu)
importlib.reload(cu)

In [ ]:
# global settings
plot_type1 = True
plot_type2 = True
plot_moment = True

# load the data
df = csvu.read_csv('data/DF2017_chargeInner_PXLayer_2.csv')
print('total number of LS: '+str(len(df)))

# first select a set of reference histograms (for plot type 2)
refhists = hu.preparedatafromdf(dfu.select_dcson(dfu.select_runs(df,[297056,297177,301449])))

# filter the data
#df = dfu.select_golden(df)
#df = dfu.select_notgolden(df)
df = dfu.select_dcson(df)
#df = dfu.select_dcsoff(df)
print('filtered number of LS: '+str(len(df)))

# start loop over runs
runs = dfu.get_runs(df)
print('number of runs: '+str(len(runs)))
runs = runs[:10]
print('will loop over following runs: '+str(runs))
for run in runs:
    print('run '+str(run))
    dfr = dfu.select_runs(df,[run])
    # get histograms
    (hists,_,ls) = dfu.get_hist_values(dfr)
    hists = hists[:,1:-1]
    # plot type 1
    if plot_type1:
        pu.plot_hists_multi(hists.tolist(),colorlist=ls)
        plt.show()
    # plot type 2
    if plot_type2:
        normhists = hu.normalizehists(hists)
        pu.plot_sets([refhists,normhists],colorlist=['blue','red'],labellist=['reference runs','this run'],transparencylist=[0.1,1.])
        plt.show()
    # get moments
    if plot_moment:
        nmoments = 3
        moments = np.zeros((len(hists),nmoments))
        xmin = 0. # some sort of normalization
        xmax = 1. # some sort of normalization
        nbins = dfr.at[0,'Xbins']
        binwidth = (xmax-xmin)/nbins
        bins = np.linspace(xmin+binwidth/2,xmax-binwidth/2,num=nbins,endpoint=True)
        for i in range(1,nmoments+1):
            moments[:,i-1] = hu.moment(bins,hists,i)
        pu.plot_moments(moments,ls,(0,1))
        dists = np.zeros(len(ls))
        for i in range(len(ls)):
            dists[i] = cu.avgnndist(moments,i,2)
        pu.plot_distance(dists,ls)
        plt.show()